# Simple subclassing model
* *__init__()* will define the layers we use
* *call()* defines the forward pass of the model

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate

class MyModel(Model):
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = Concatenate()
        self.mixing_layer = Dense(64, activation="relu")
        self.priority_scorer = Dense(1, activation="sigmoid", name="priority")
        self.department_classifier = Dense(num_departments, activation="softmax", name="department")

    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        # forward pass
        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)

        return priority, department

In [2]:
import numpy as np

# create random data
num_samples = 1280
num_departments = 4
vocabulary_size = 10_000
num_tags = 100

# input data -> X
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size)) # size (batch, data-shape)
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

# targets -> y
priority_targets = np.random.random(size=(num_samples,1))
department_targets = np.random.random(size=(num_samples, num_departments))

In [3]:
model = MyModel(num_departments=num_departments)

In [4]:
# built its weights
model({"title": title_data, "text_body": text_body_data, "tags": tags_data})

(<tf.Tensor: shape=(1280, 1), dtype=float32, numpy=
 array([[0.11610715],
        [0.3059407 ],
        [0.41626173],
        ...,
        [0.2263044 ],
        [0.2861188 ],
        [0.51231235]], dtype=float32)>,
 <tf.Tensor: shape=(1280, 4), dtype=float32, numpy=
 array([[0.20962095, 0.24272539, 0.3691065 , 0.17854716],
        [0.09454552, 0.0857659 , 0.5417931 , 0.27789545],
        [0.3094408 , 0.09544255, 0.32890728, 0.26620936],
        ...,
        [0.14091545, 0.21534704, 0.5043391 , 0.13939841],
        [0.13843128, 0.3396377 , 0.34319004, 0.17874096],
        [0.31407478, 0.08921625, 0.2914976 , 0.30521145]], dtype=float32)>)

In [9]:
model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

In [10]:
model.fit(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data},
    [priority_targets, department_targets],
    epochs=1
)

40/40 [==============================] - 2s 6ms/step - loss: 35.0853 - output_1_loss: 0.3163 - output_2_loss: 34.7690 - output_1_mean_absolute_error: 0.4818 - output_2_accuracy: 0.2688
